In [5]:
# Imports
from sklearn.neural_network import MLPClassifier
import numpy as np
import time
import pandas as pd
import nltk
from nltk.stem.lancaster import LancasterStemmer
from math import isnan
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import string
from sklearn.svm import SVC

stemmer = LancasterStemmer()

* topic detection literature - extract features of plain text.
* spelling correction
* narrow down to manual grading. Flagging if existing words in the string are below a system of certainty.

In [7]:
answers = pd.read_excel('litmus_results_trial1.xlsx').set_index('student_id')

In [12]:
answers.groupby('class_code').nunique()['Type'][answers.groupby('class_code').nunique()['Type']>1]

class_code
FKYGSM    2
MH6FDC    2
Name: Type, dtype: int64

In [3]:


answers_subset = answers.dropna(how='all',subset=['question_id_11459_answer',
                                                  'question_id_11460_answer',
                                                  'question_id_11461_answer'])

answers_graded = answers.dropna(how='any',subset=['question_id_11459_score',
                                                  'question_id_11460_score',
                                                  'question_id_11461_score'])

In [3]:
answers_graded.loc[:,'question_id_11459_score'] = answers_graded['question_id_11459_score'].astype('str')
answers_graded.loc[:,'question_id_11460_score'] = answers_graded['question_id_11460_score'].astype('str')
answers_graded.loc[:,'question_id_11461_score'] = answers_graded['question_id_11461_score'].astype('str')


/Users/sarangof/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [3]:
X_train_59, X_test_59, y_train_59, y_test_59 = train_test_split(answers_graded['question_id_11460_answer'], 
                                                        answers_graded['question_id_11460_score'], 
                                                        test_size=0.2, 
                                                        random_state=42)

X_train_60, X_test_60, y_train_60, y_test_60 = train_test_split(answers_graded['question_id_11460_answer'], 
                                                        answers_graded['question_id_11460_score'], 
                                                        test_size=0.2, 
                                                        random_state=42)

X_train_61, X_test_61, y_train_61, y_test_61 = train_test_split(answers_graded['question_id_11460_answer'], 
                                                        answers_graded['question_id_11460_score'], 
                                                        test_size=0.2, 
                                                        random_state=42)

In [12]:
len(answers)

2824

In [13]:
len(answers_graded)

243

In [14]:
len(answers_subset)

1631

In [15]:
def transform_na(text):
    try: 
        if isnan(text):
            return ''
        else:
            return unicode(text)
    except TypeError:
        return unicode(text)

training_data_59 = [{'score':x,'text':transform_na(y)} for x,y in zip(answers_graded.loc[X_train_59.index.values]['question_id_11459_score'].values,
                                                                      answers_graded.loc[X_train_59.index.values]['question_id_11459_answer'].values)]

testing_data_59 = [{'score':x,'text':transform_na(y)} for x,y in zip(answers_graded.loc[X_test_59.index.values]['question_id_11459_score'].values,
                                                                     answers_graded.loc[X_test_59.index.values]['question_id_11459_answer'].values)]

training_data_60 = [{'score':x,'text':transform_na(y)} for x,y in zip(answers_graded.loc[X_train_60.index.values]['question_id_11460_score'].values,
                                                                      answers_graded.loc[X_train_60.index.values]['question_id_11460_answer'].values)]

testing_data_60 = [{'score':x,'text':transform_na(y)} for x,y in zip(answers_graded.loc[X_test_60.index.values]['question_id_11460_score'].values,
                                                                     answers_graded.loc[X_test_60.index.values]['question_id_11460_answer'].values)]

training_data_61 = [{'score':x,'text':transform_na(y)} for x,y in zip(answers_graded.loc[X_train_61.index.values]['question_id_11461_score'].values,
                                                                      answers_graded.loc[X_train_61.index.values]['question_id_11461_answer'].values)]

testing_data_61 = [{'score':x,'text':transform_na(y)} for x,y in zip(answers_graded.loc[X_test_61.index.values]['question_id_11461_score'].values,
                                                                     answers_graded.loc[X_test_61.index.values]['question_id_11461_answer'].values)]

In [16]:
ignore_words = list(set(list(string.punctuation))-set(['?','"'])) #+ ['?'] +  stopwords.words('english')

def define_elements(training_data):
    words = []
    classes = []
    documents = []
    for pattern in training_data:
        w_pre = []
        split_words = pattern['text'].split(" ")
        for wrd in split_words:
            wrd = "".join(wrd.split("'"))
            wrd = "".join(wrd.split("`"))
            wrd = "".join(wrd.split("-"))
            wrd = "".join(wrd.split("`"))
            w_pre.append(wrd)
            
        #w = nltk.word_tokenize(w_pre)
        # add to our words list
        words.extend(w_pre)
        # add to documents in our corpus
        documents.append((w_pre, pattern['score']))
        # add to our classes list
        if pattern['score'] not in classes:
            classes.append(pattern['score'])
    return words,documents,classes

words_59, documents_59, classes_59 = define_elements(training_data_59)
words_60, documents_60, classes_60 = define_elements(training_data_60)
words_61, documents_61, classes_61 = define_elements(training_data_61)

# stem and lower each word and remove duplicates
#words_59 = [stemmer.stem(w.lower()) for w in words_59 if w not in ignore_words]
words_59 = [w.lower() for w in words_59 if w not in ignore_words]
words_59 = list(set(words_59))
#words_60 = [stemmer.stem(w.lower()) for w in words_60 if w not in ignore_words]
words_60 = [w.lower() for w in words_60 if w not in ignore_words]
words_60 = list(set(words_60))
#words_61 = [stemmer.stem(w.lower()) for w in words_61 if w not in ignore_words]
words_61 = [w.lower() for w in words_61 if w not in ignore_words]
words_61 = list(set(words_61))

# remove duplicates
classes_59 = list(set(classes_59))
classes_60 = list(set(classes_60))
classes_61 = list(set(classes_61))

#print (len(documents), "documents")
#print (len(classes), "classes", classes)
#print (len(words), "unique stemmed words", words)

In [17]:
words_59

[u'',
 u'go',
 u'young',
 u'passage',
 u'to',
 u'anddancing',
 u'dressing',
 u'mother,',
 u'song',
 u'very',
 u'trouble',
 u'begening',
 u'danicing.',
 u'her,"why',
 u'makebelieve[par.1]',
 u'nine.',
 u'ups',
 u'talked',
 u'says',
 u'her.',
 u'her,',
 u'bluederes',
 u'purtending',
 u'blue',
 u'believe,and',
 u'what',
 u'holdiding',
 u'version',
 u'told',
 u'hey,monica',
 u'here',
 u'hers',
 u'sing',
 u'great',
 u'kids',
 u'blueberrys',
 u'my',
 u'childeren]',
 u'arent',
 u'pick',
 u'store."',
 u'makebalieve',
 u'love',
 u'suddenly',
 u'stop/quit.',
 u'kichion',
 u'\u201cyeah\u2026but',
 u'singing',
 u'story.',
 u'story,',
 u'believe?',
 u'would',
 u'"arent',
 u'right?',
 u'few',
 u'believe.',
 u'music',
 u'makebelive."',
 u'right.',
 u'type',
 u'blueberry',
 u'singing.',
 u'phone',
 u'this',
 u'pretend',
 u'movies',
 u'can',
 u'making',
 u'didnt',
 u'that\xb4s',
 u'monica',
 u'makebe?"',
 u'allowed',
 u'bielive',
 u'end',
 u'said,',
 u'begging',
 u'after',
 u'tipped',
 u'mad',
 u'mam',

In [18]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
# convert output of sigmoid function to its derivative -- should I?

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    #sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    sentence_words = [word.lower() for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

# Question 59

In [20]:
in_list_59 = []
for x in training_data_59:
    sentence = x['text']
    in_list_59.append(bow(sentence.lower(),words_59))

In [23]:
len(words_59)

357

In [24]:
len(words_60)

357

In [25]:
len(words_61)

684

In [21]:
# samples, features. 121 sammples. 214 features.
clf_59 = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(50, 3), random_state=1)
clf_59.fit(np.array(in_list_59),np.array([x['score'] for x in training_data_59])) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

### Training

In [12]:
train_list_59 = []
for x in training_data_59:
    sentence = x['text']
    train_list_59.append(bow(sentence.lower(),words_59))
    
float(sum([x==y for x,y in zip(clf_59.predict(np.array(train_list_59)),y_train_59.values)]))/len(y_train_59)

0.5824742268041238

In [13]:
svm_59 = SVC()

svm_59.fit(np.array(in_list_59),np.array([x['score'] for x in training_data_59])) 
float(np.sum([x==y for x,y in zip(svm_59.predict(np.array(train_list_59)),y_train_59.values)]))/len(y_train_59.values)

0.5979381443298969

### Testing

In [14]:
# NN
test_list_59 = []
for x in testing_data_59:
    sentence = x['text']
    test_list_59.append(bow(sentence.lower(),words_59))
float(np.sum([x==y for x,y in zip(clf_59.predict(np.array(test_list_59)),y_test_59.values)]))/len(y_test_59.values)

0.6326530612244898

In [15]:
# SVM
float(np.sum([x==y for x,y in zip(svm_59.predict(np.array(test_list_59)),y_test_59.values)]))/len(y_test_59.values)

0.673469387755102

In [16]:
clf_59.predict(bow("We're WHAT? What am I? Slave labor?", words_59).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

In [17]:
clf_59.predict(bow("Arent you old to play make-believe?", words_59).reshape(1, -1))

array(['0.75'], 
      dtype='|S4')

In [18]:
clf_59.predict(bow("Monica asks, “‘Arent you a little old to be playing make-believe?’”",words_59).reshape(1, -1))

/Users/sarangof/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


array(['1.0'], 
      dtype='|S4')

In [19]:
clf_59.predict(bow("Does not say anything",words_59).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

In [20]:

svm_59.predict(bow("We're WHAT? What am I? Slave labor?", words_59).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

In [21]:
svm_59.predict(bow("Arent you old to play make-believe?", words_59).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [22]:
svm_59.predict(bow("Monica asks, “‘Arent you a little old to be playing make-believe?’”",words_59).reshape(1, -1))

/Users/sarangof/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


array(['1.0'], 
      dtype='|S4')

In [23]:
svm_59.predict(bow("Does not say anything",words_59).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

# Question 60

In [24]:
in_list_60 = []
for x in training_data_60:
    sentence = x['text']
    in_list_60.append(bow(sentence.lower(),words_60))

In [25]:
# samples, features. 121 sammples. 214 features.
clf_60 = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(500, 30), random_state=1)
clf_60.fit(np.array(in_list_60),np.array([x['score'] for x in training_data_60])) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

### Training

In [26]:
train_list_60 = []
for x in training_data_60:
    sentence = x['text']
    train_list_60.append(bow(sentence.lower(),words_60))
float(np.sum([x==y for x,y in zip(clf_60.predict(np.array(train_list_60)),y_train_60.values)]))/len(y_train_60.values)

1.0

In [27]:
svm_60 = SVC()

svm_60.fit(np.array(in_list_60),np.array([x['score'] for x in training_data_60])) 
float(np.sum([x==y for x,y in zip(svm_60.predict(np.array(train_list_60)),y_train_60.values)]))/len(y_train_60.values)

0.6494845360824743

In [28]:
#for x,y,z in zip(clf_60.predict(np.array(train_list_60)),y_train_60.values,[zz['text'] for zz in training_data_60]):
#    print(y,x,z)

### Testing

In [29]:
test_list_60 = []
for x in testing_data_60:
    sentence = x['text']
    test_list_60.append(bow(sentence.lower(),words_60))
float(np.sum([x==y for x,y in zip(clf_60.predict(np.array(test_list_60)),y_test_60.values)]))/len(y_test_60.values)

0.7755102040816326

In [30]:
float(np.sum([x==y for x,y in zip(svm_60.predict(np.array(test_list_60)),y_test_60.values)]))/len(y_test_60.values)

0.6938775510204082

In [31]:
clf_60.predict(bow("She is happy",words_60).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

In [32]:
clf_60.predict(bow("She smiles",words_60).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [33]:
clf_60.predict(bow("Monica smiles",words_60).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [34]:
svm_60.predict(bow("She is happy",words_60).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [35]:
svm_60.predict(bow("She smiles",words_60).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [36]:
svm_60.predict(bow("Monica smiles",words_60).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

# Question 61

In [37]:
in_list_61 = []
for x in training_data_61:
    sentence = x['text']
    in_list_61.append(bow(sentence.lower(),words_61))

In [38]:
# samples, features. 121 sammples. 214 features.
clf_61 = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(50, 30), random_state=1)
clf_61.fit(np.array(in_list_61),np.array([x['score'] for x in training_data_61])) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [39]:
#SVM
svm_61 = SVC(kernel='poly')
#‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’

### Training

In [40]:
train_list_61 = []
for x in training_data_61:
    sentence = x['text']
    train_list_61.append(bow(sentence.lower(),words_61))
float(np.sum([x==y for x,y in zip(clf_61.predict(np.array(train_list_61)),y_train_61.values)]))/len(y_train_61.values)

0.5567010309278351

In [41]:
svm_61.fit(np.array(in_list_61),np.array([x['score'] for x in training_data_61])) 
float(np.sum([x==y for x,y in zip(svm_61.predict(np.array(train_list_61)),y_train_61.values)]))/len(y_train_61.values)

0.6494845360824743

### Testing

In [42]:
test_list_61 = []
for x in testing_data_61:
    sentence = x['text']
    test_list_61.append(bow(sentence.lower(),words_61))
    
float(np.sum([x==y for x,y in zip(clf_61.predict(np.array(test_list_61)),y_test_61.values)]))/len(y_test_61.values)

0.4489795918367347

In [43]:
test_list_61 = []
for x in testing_data_61:
    sentence = x['text']
    test_list_61.append(bow(sentence.lower(),words_61))
    
float(np.sum([x==y for x,y in zip(svm_61.predict(np.array(test_list_61)),y_test_61.values)]))/len(y_test_61.values)

0.6938775510204082

In [44]:
clf_61.predict(bow("",words_61).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

In [45]:
clf_61.predict(bow("Monica's response changed because of what happened when she went blueberry picking with her mother. She started having fun when she was singing and dancing under the branches. So she understood why her mom sang and danced for fun at the end of the story. Also, Monica got stung by a bee, and her mom took care of her by hugging her and giving her first aid cream. So at the end of the story, she wasn't annoyed at her mom anymore.",words_61).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [46]:
clf_61.predict(bow("Monica smiled",words_61).reshape(1, -1))

array(['0.0'], 
      dtype='|S4')

In [47]:
svm_61.predict(bow("Monica's response changed because of what happened when she went blueberry picking with her mother. She started having fun when she was singing and dancing under the branches. So she understood why her mom sang and danced for fun at the end of the story. Also, Monica got stung by a bee, and her mom took care of her by hugging her and giving her first aid cream. So at the end of the story, she wasn't annoyed at her mom anymore.",words_61).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [48]:
svm_61.predict(bow("Monica smiled",words_61).reshape(1, -1))

array(['1.0'], 
      dtype='|S4')

In [49]:
X_test_61[[x==y for x,y in zip(clf_61.predict(np.array(test_list_61)),y_test_61.values)]].values

array([u'take a nap.', u'hums west side story',
       u'Monica smiles at her mom ', u'she sckooped ice cream',
       u'Instead of grumbling about the song, Monica smiles, and goes to sleep.',
       u'when Monica hears her mom singing to West Side Story she smiles.',
       u'Sing along.', u'Monica smiles', u'Anorse it and say mene stuff',
       u'When Monica hears her mother singing along to West side story. (at the end.)Monica smiles.',
       u'when Monica hears her mother singing along to west side story at the end of the story the thing she does is she starts to smile.',
       u'monico starts humming the song her mom was singing',
       u'when her mother started to sing againthis time she just smiled.',
       u'Monica to west side story and does exactly what her mother does.',
       u'shegaveheraid creamforthefirsttime',
       u'The thing Monica does when she hears her mother singing along to West Side Story at the end of the story is smile.',
       u'when Monica hearsher

In [50]:
y_test_61[[x==y for x,y in zip(svm_61.predict(np.array(test_list_61)),y_test_61.values)]].values

array(['1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0'], dtype=object)

In [51]:
X_test_61[[x!=y for x,y in zip(svm_61.predict(np.array(test_list_61)),y_test_61.values)]].values

array([ u'When she hears her mother singing along to West Side Story at the end of the story Monica starts singing and dancing.',
       u'hums west side story', u'monica starts singing the same song.',
       u'she sckooped ice cream', u'she sings along ',
       u'\u201cAren\u2019t you a little old to be playing make-believe?\u201dI, myself, had given up on play-acting when I was nine. It was kid\u2019s stuff, and I didn\u2019t feel very much like a kid anymore. My mother, however, seemed to have absorbed everything about childhood that I\u2019d left behind. Now she was dancing around the living room with the handle of a dust mop in her hand, held at an angle like a rock star\u2019s microphone, singing.When I said that, though, she stopped.\u201cHey, Monica, you like movies, right? Some people make a living out of playing make-believe.\u201dShe wasn\u2019t wrong. I did like movies, and actors did make a living dressing up, and pretending they were someone else. The fact that Mom was 

In [52]:
svm_61.predict(np.array(test_list_61))

array(['1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0'], 
      dtype='|S4')

In [53]:
clf_61.predict(np.array(test_list_61))

array(['1.0', '1.0', '0.0', '0.25', '0.5', '1.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.5', '1.0', '1.0', '1.0', '0.0',
       '0.0', '1.0', '0.0', '1.0', '0.0', '1.0', '0.5', '0.0', '1.0',
       '1.0', '1.0', '0.5', '0.5', '0.0', '1.0', '1.0', '0.5', '0.25',
       '1.0', '0.5', '0.0', '0.5', '0.5', '0.0', '1.0', '0.0', '1.0',
       '1.0', '1.0', '0.0', '1.0'], 
      dtype='|S4')

In [54]:
y_test_61[[x!=y for x,y in zip(svm_61.predict(np.array(test_list_61)),y_test_61.values)]].values

array(['0.5', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.75', '0.0', '0.0', '0.0', '0.0', '0.0'], dtype=object)

In [55]:
svm_61.predict(np.array(test_list_61))

array(['1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '1.0'], 
      dtype='|S4')

In [57]:
words_59

[u'bielive',
 u'mom',
 u'just',
 u'being',
 u'text',
 u'over',
 u'paragraph',
 u'beleve',
 u'go',
 u'make-belieive',
 u'yes',
 u'probbly',
 u'trouble',
 u'help',
 u'beleive',
 u'makebeileve',
 u'thinking',
 u'harvest',
 u'had',
 u'young',
 u'passage',
 u'better',
 u'to',
 u'going',
 u'believing',
 u'dressing',
 u'playingmake-believe',
 u'jumped',
 u'he',
 u'into',
 u'loved',
 u'morther',
 u'monica',
 u'them',
 u'someone',
 u'around',
 u'disrespectful',
 u'get',
 u'very',
 u'stung',
 u'mop',
 u'stop',
 u'stor',
 u'begging',
 u'\u201ci',
 u'par.1',
 u'storey',
 u'they',
 u'not',
 u'labor',
 u'bit',
 u'now',
 u'begening',
 u'is.hermotherlovewestsidestory',
 u'like',
 u'playin',
 u'stop/quit',
 u"are'nt",
 u'berrys',
 u'liks',
 u'stopped',
 u'she',
 u'rock',
 u'told',
 u'where',
 u'side',
 u'likes',
 u'few',
 u'right',
 u'old',
 u"''are",
 u"''now",
 u'doing',
 u'house',
 u'danicing',
 u'some',
 u'make-balieve',
 u'playmake',
 u'see',
 u'are',
 u'star',
 u'purtending',
 u'littlie',
 u'best